In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("drive/My Drive/QCM GENERATION AI/squad.csv")
contexts = pd.read_csv("drive/My Drive/QCM GENERATION AI/contexts.csv")

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
answer = "he ate it"
question = "what did nazim do to the apple ?"
context  = "nazim ate the apple"

answer_tokens = tokenizer.tokenize(answer)
answer_token_ids = tokenizer.convert_tokens_to_ids(answer_tokens)

question_tokens = tokenizer.tokenize(question)
question_token_ids = tokenizer.convert_tokens_to_ids(question_tokens)

context_tokens = tokenizer.tokenize(context)
context_token_ids = tokenizer.convert_tokens_to_ids(context_tokens)

print(f'   Answer: {answer}')
print(f'   Tokens: {answer_tokens}')
print(f'Token IDs: {answer_token_ids}', end="\n\n")

print(f' Question: {question}')
print(f'   Tokens: {question_tokens}')
print(f'Token IDs: {question_token_ids}', end="\n\n")

print(f'  Context: {context}')
print(f'   Tokens: {context_tokens}')
print(f'Token IDs: {context_token_ids}', end="\n\n")

   Answer: he ate it
   Tokens: ['he', 'ate', 'it']
Token IDs: [1119, 8756, 1122]

 Question: what did nazim do to the apple ?
   Tokens: ['what', 'did', 'na', '##zi', '##m', 'do', 'to', 'the', 'apple', '?']
Token IDs: [1184, 1225, 9468, 5303, 1306, 1202, 1106, 1103, 12075, 136]

  Context: nazim ate the apple
   Tokens: ['na', '##zi', '##m', 'ate', 'the', 'apple']
Token IDs: [9468, 5303, 1306, 8756, 1103, 12075]



In [ ]:
context_answer_seq = "[CLS] " + context + " [SEP] " + answer + " [SEP] "
context_answer_seq_tokens = tokenizer.tokenize(context_answer_seq)
context_answer_seq_token_ids = tokenizer.convert_tokens_to_ids(context_answer_seq_tokens)

print(context_answer_seq)
print(context_answer_seq_tokens)
print(context_answer_seq_token_ids)

question_seq_tokens = tokenizer.tokenize(question)
question_seq_token_ids = tokenizer.convert_tokens_to_ids(question_seq_tokens)

print(question)
print(question_seq_tokens)
print(question_seq_token_ids)

question_seq_token_ids.append(102)

X_token_ids = []
Y_token_ids = []
for i in range(len(question_seq_token_ids)):
    X_token_ids.append(context_answer_seq_token_ids + question_seq_token_ids[:i] + [103])
    Y_token_ids.append(question_seq_token_ids[i])

[CLS] nazim ate the apple [SEP] he ate it [SEP] 
['[CLS]', 'na', '##zi', '##m', 'ate', 'the', 'apple', '[SEP]', 'he', 'ate', 'it', '[SEP]']
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102]
what did nazim do to the apple ?
['what', 'did', 'na', '##zi', '##m', 'do', 'to', 'the', 'apple', '?']
[1184, 1225, 9468, 5303, 1306, 1202, 1106, 1103, 12075, 136]


In [ ]:
for i in range(len(Y_token_ids)):
    print(X_token_ids[i], "---->", Y_token_ids[i])
    

[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 103] ----> 1184
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 103] ----> 1225
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 103] ----> 9468
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 9468, 103] ----> 5303
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 9468, 5303, 103] ----> 1306
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 9468, 5303, 1306, 103] ----> 1202
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 9468, 5303, 1306, 1202, 103] ----> 1106
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 9468, 5303, 1306, 1202, 1106, 103] ----> 1103
[101, 9468, 5303, 1306, 8756, 1103, 12075, 102, 1119, 8756, 1122, 102, 1184, 1225, 9468, 5303, 1306, 1202, 1106, 1103, 103] ----> 12075


In [ ]:
def get_Xseq_Y_from_CAQ(context, answer, question):
    """
        exemple: [CLS] + context + [SEP] + answer [SEP] ---> [101, a, b, c, d, 102, e, f, g, 102]
                 question ---> [h, i, j, k]

                 outputs: X_token_ids = [ [101, a, b, c, d, 102, e, f, g, 102, 103]
                                          [101, a, b, c, d, 102, e, f, g, 102, h, 103]
                                          [101, a, b, c, d, 102, e, f, g, 102, h, i, 103] 
                                          [101, a, b, c, d, 102, e, f, g, 102, h, i, j, 103]
                                          [101, a, b, c, d, 102, e, f, g, 102, h, i, j, k, 103] ]
                          Y_token_ids = [ h, i, j, k, 102 ]
    
    """
    context_answer_seq = "[CLS] " + context + " [SEP] " + answer + " [SEP] "
    context_answer_seq_tokens = tokenizer.tokenize(context_answer_seq)
    context_answer_seq_token_ids = tokenizer.convert_tokens_to_ids(context_answer_seq_tokens)


    question_seq_tokens = tokenizer.tokenize(question)
    question_seq_token_ids = tokenizer.convert_tokens_to_ids(question_seq_tokens)

    question_seq_token_ids.append(102)


    X_token_ids = []
    Y_token_ids = []
    for i in range(len(question_seq_token_ids)):
        X_token_ids.append(context_answer_seq_token_ids + question_seq_token_ids[:i] + [103])
        Y_token_ids.append(question_seq_token_ids[i])

    return (X_token_ids, Y_token_ids)

In [ ]:
all_the_X = []
all_the_Y = []

for i in range(len(data)):
    try:
        sample = data.iloc[i]
        question = sample["questions"]
        answer   = sample["answers"]
        context  = contexts.iloc[int(sample["contexts"])]["contexts_texts"]
        X, Y = get_Xseq_Y_from_CAQ(context, answer, question)
        for x, y in zip(X, Y):
            all_the_X.append(x)
            all_the_Y.append(y)

        print(i,"/",len(data)) 
    except:
        break

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
102679 / 130319
102680 / 130319
102681 / 130319
102682 / 130319
102683 / 130319
102684 / 130319
102685 / 130319
102686 / 130319
102687 / 130319
102688 / 130319
102689 / 130319
102690 / 130319
102691 / 130319
102692 / 130319
102693 / 130319
102694 / 130319
102695 / 130319
102696 / 130319
102697 / 130319
102698 / 130319
102699 / 130319
102700 / 130319
102701 / 130319
102702 / 130319
102703 / 130319
102704 / 130319
102705 / 130319
102706 / 130319
102707 / 130319
102708 / 130319
102709 / 130319
102710 / 130319
102711 / 130319
102712 / 130319
102713 / 130319
102714 / 130319
102715 / 130319
102716 / 130319
102717 / 130319
102718 / 130319
102719 / 130319
102720 / 130319
102721 / 130319
102722 / 130319
102723 / 130319
102724 / 130319
102725 / 130319
102726 / 130319
102727 / 130319
102728 / 130319
102729 / 130319
102730 / 130319
102731 / 130319
102732 / 130319
102733 / 130319
102734 / 130319
102735 / 130319
102736 / 1